## 1.1-Importando librerias necesarias.

In [1]:
from gensim.models import KeyedVectors
import matplotlib.pyplot as  plt
from collections import Counter
import pandas as pd
import numpy as np
import random
import string
import math
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

/home/fedricio/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1-Dataset Puestos

In [2]:
df_Jobs = pd.read_csv("Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/Job_Descr/Job_Positions_Scrapped/10_Positions_Only_Qualif.csv")
df_Jobs = df_Jobs.rename(columns={'job_title': 'Job_Title', 'job_description':'Job_Description'})

df_Jobs = df_Jobs.sort_values('Job_Title', ascending=True)
df_Jobs

,Job_Title,Job_Description
2,Data Scientist,"Master’s degree or above in a STEM field, incl..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...
7,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...
0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...
8,Security Specialist,Work in a fast-paced environment that combine ...
9,Security Specialist 2,\n Handling incoming requests for assistanc...
4,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...
5,Web Developer Full Stack 2,\n· Enter existing website codebases and exten...


### 2-Dataset CVs

### PDF a texto usando pdfplumber.

In [3]:
import pdfplumber
import os
import collections
from os import listdir
from os.path import isfile, join

#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno convirtiendolos a texto
#mediante la libreria pdfplumber:
pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs_2'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]
print("Cantidad de CVs extraidos:", len(onlyfiles))

#Funcion para extraer las palabras del CV:

def pdfextract(PDF_file):
    single_page_text = ""
    all_text = ""
    pdf = pdfplumber.open(PDF_file)
    for pdf_page in pdf.pages:
        single_page_text = pdf_page.extract_text()
        all_text = all_text + '\n' + single_page_text
    pdf.close()
    #return(all_text)        
    return(all_text.encode('utf-8'))

def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    #text = pdfextract(file)
    return text

Cantidad de CVs extraidos: 10


In [4]:
#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates = df_Candidates.sort_values('Candidate_Name', ascending=True)
df_Candidates

,Candidate_Name,Content_CV
6,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...
2,Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...
4,HCM_Federico_Calonge,\n ...
7,HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...
0,ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+..."
1,ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n..."
8,Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation..."
3,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...
5,Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io..."
9,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...


**Limpieza De los CVs**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza en txt de datos personales (mails, telefonos).
6. Limpieza de palabras innecesarias en archivo txt (Page, meses, años, etc).

**Limpieza de los Puestos**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza de palabras innecesarias en archivo txt (VER: por ej. mails que dejan las empresas, telefonos, etc.).
6. Luego, para la obtención de Keywords (VER si es necesario): --> Por el problema que me tomaba las keywords como "you", "I" y demás..., podría hacer un txt con lenguajes y skills por "defecto" y me fijo si están en las keywords... sino no las pongo.


### Funciones para limpiar datos en columnas de un DF

In [5]:
import regex as re #Usado en la función remove_punctuation_and_special_characters.
from nltk.stem import WordNetLemmatizer   #Usado para lematización.
nltk.download('wordnet')
from nltk.tokenize import word_tokenize   #Usado para tokenizar.

#Definimos nuestras stop words desde nltk:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
                
def lower_text(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: x.lower() if isinstance(x,str) else x)
    
def remove_stop_words(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

def remove_punctuation_and_special_characters(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: re.sub('[^\w-]|(\d+)',' ', x) if isinstance(x,str) else x)
                                                                #Macheamos cualquier non-word excepto el -; y tambien macheamos los números (\d+)
    #Macheos:
        #'\W+'                  --> Cualquier non-word.
        #'[()!@#$/,;.:"]'       --> todos esos carácteres especiales que pongo ahi. 
        #'\w+[-]\w+'            --> machea a las palabras non-algo.
        #'[^\w-+]'              --> Machea todas las non-words excepto el guion medio y el +.
        #'\C\+\+'               --> Machea C++.
        #[^\w-+]|(\C\+\+|team)  --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y la palabra "team"
        #[^\w-+]|(\C\+\+|\d+)   --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y los números (\d+).
        
    #Para m.sc' --> reemplazar el . por espacio cuando antes y despues del punto esta todo "junto"... para diferenciarlo del punto de una oración. 
    
#Tokenizar es el proceso de parsear Strings de texto en diferentes secciones ("tokens"). Debemos tokenizar teniendo en cuenta qué hacer con las comas, con los espacios en blanco, 
#con los "-", etc. Para esto usamos la función "word_tokenize" que se encargará de esta tarea. 
#Esta funcion NO divide las palabras que estan con un "-"... de esta manera la palabra "scale-up" queda así y NO se divide.
#Tambien tendremos la palabra con una única letra ("c", del lenguaje c) macheado únicamente a un token "c". 
def tokenize_and_lemmatization(text_column):
    tokens = word_tokenize(text_column) #Tokenizamos.
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(tok).lower() for tok in tokens] # Stem words.
    return list(lemmatized_tokens)
    
def cleaning_DF(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    DF[clean_column]=DF[column_to_clean] #Copiamos el contenido de 'column_to_clean' en 'clean_column' para utilizarla en las funciones posteriores.
    lower_text(DF,clean_column)
    remove_punctuation_and_special_characters(DF,clean_column)
    remove_stop_words(DF,clean_column)
     
def tokenize_and_lemmatize(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    tokens_column='tokens_'+column_to_clean
    DF[tokens_column] = DF[clean_column].apply(tokenize_and_lemmatization)

[nltk_data] Downloading package wordnet to /home/fedricio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Limpieza de datos del DF de Jobs.

In [6]:
cleaning_DF(df_Jobs,'Job_Description')
tokenize_and_lemmatize(df_Jobs,'Job_Description')


#Tomamos el job_description de la posición 5 como ejemplo (Machine Learning Engineer 2'):

print(df_Jobs['Job_Description'].iloc[5])  
#non-expert  debería mantenerse.
#deberia juntar el Computer Science
print("#######################################################")
print(df_Jobs['clean_Job_Description'].iloc[5]) 
print("#######################################################")
print(df_Jobs['tokens_Job_Description'].iloc[5]) 
print("#######################################################")

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

In [7]:
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
2,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
7,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp..."
0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
8,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
9,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."
4,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."
5,Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ..."


#### Limpieza de datos del DF de Candidatos.

In [8]:
cleaning_DF(df_Candidates,'Content_CV')
tokenize_and_lemmatize(df_Candidates,'Content_CV')

df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
6,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
2,Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
4,HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co..."
7,HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
0,ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
8,Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
3,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
5,Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."
9,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."


In [9]:
#Vemos los tokens generados para el candidato HCM_Federico_Calonge:
df_Candidates.iloc[2]['tokens_Content_CV']

['calonge',
 'federico',
 'matías',
 'hcm',
 'technical',
 'consultant',
 'federico',
 'working',
 'oracle',
 'tool',
 'since',
 'october',
 'march',
 'participated',
 'erp',
 'cloud',
 'project',
 'performing',
 'reporting',
 'task',
 'working',
 'module',
 'ap',
 'ar',
 'gl',
 'since',
 'march',
 'participating',
 'hcm',
 'cloud',
 'project',
 'performing',
 'reporting',
 'extraction',
 'integration',
 'task',
 'working',
 'module',
 'core',
 'recruitment',
 'since',
 'october',
 'member',
 'artificial',
 'intelligence',
 'ai',
 'committee',
 'oracle',
 'strong',
 'interest',
 'data',
 'science',
 'machine',
 'learning',
 'last',
 'year',
 'computer',
 'engineering',
 'degree',
 'studying',
 'th',
 'year',
 'english',
 'personal',
 'academic',
 'programming',
 'project',
 'http',
 'github',
 'com',
 'federicocalonge',
 'skill',
 'summary',
 'oracle',
 'hcm',
 'cloud',
 'core',
 'recruitment',
 'oracle',
 'erp',
 'cloud',
 'account',
 'receivable',
 'ar',
 'account',
 'payable',
 'ap'

### AGREGADO: Funciones para tokenizar textos de una columna de un DF

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import string
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# ---------------------------------------------------------------

In [31]:

#FUNCION QUE USABA ANTES.

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

def text_to_vector_TF_IDF(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.


#REAL:

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

col_one_list = df_Jobs['Job_Description'].tolist()
corpus = col_one_list

from sklearn.metrics.pairwise import cosine_similarity

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# compute and print the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)


def get_cosine(doc1, doc2):
    #Get the cosine similarity between two documents.
    #Depends on the angle between two non zero vectors which are constructed by each word frequency in the two documents.
        #input  --> doc1: the first document as STRING.
                    #doc2: the second document as STRING.
        #output --> cosine similarity score
    
    #vec1 = text_to_vector([doc1])
    #print(vec1)
    #vec2 = text_to_vector([doc2])
   
    vec1 = text_to_vector_TF_IDF([doc1])
    #print(vec1)
    vec2 = text_to_vector_TF_IDF([doc2])
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [32]:
first_job_descr = df_Jobs.loc[0,'clean_Job_Description']
first_content_cv = df_Candidates.loc[0,'clean_Content_CV']

cosine_result = round(get_cosine(first_job_descr,first_content_cv),3)
cosine_result

Counter({'business': 6, 'learning': 5, 'data': 4, 'experience': 4, 'machine': 3, 'deep': 3, 'across': 3, 'cloud': 3, 'ability': 3, 'practices': 2, 'units': 2, 'unit': 2, 'performance': 2, 'new': 2, 'development': 2, 'technical': 2, 'organization': 2, 'managing': 2, 'algorithms': 2, 'tools': 2, 'etc': 2, 'industry': 2, 'computer': 2, 'engineering': 2, 'proficient': 2, 'knowledge': 2, 'environment': 2, 'aws': 2, 'gcp': 2, 'projects': 2, 'highly': 2, 'skills': 2, 'leveraging': 1, 'latest': 1, 'techniques': 1, 'challenge': 1, 'current': 1, 'enhancing': 1, 'internal': 1, 'collaboration': 1, 'alignment': 1, 'people': 1, 'processes': 1, 'technologies': 1, 'goals': 1, 'proposing': 1, 'best': 1, 'optimize': 1, 'presence': 1, 'security': 1, 'cost': 1, 'develop': 1, 'specs': 1, 'documentation': 1, 'partake': 1, 'procedures': 1, 'user': 1, 'support': 1, 'guides': 1, 'carry': 1, 'debugging': 1, 'troubleshooting': 1, 'modifications': 1, 'testing': 1, 'custom': 1, 'solutions': 1, 'built': 1, 'platfor

0.478

In [ ]:
#AHORA:

In [64]:
#Nos quedarán 10 x 10 = 100 filas.
df_Jobs_and_Candidates = pd.merge(df_Candidates.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,tfidf_cosine,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",1.0,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
1,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",1.0,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
2,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",1.0,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",1.0,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp..."
4,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",1.0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ..."
...,...,...,...,...,...,...,...,...,...
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",1.0,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",1.0,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",1.0,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",1.0,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."


In [83]:
#https://www.py4u.net/discuss/188191
    
#Calculate cosine similarity using tf-idf vectors, between search queries and matched documents.
#Acá mi search query es la columna clean_Content_CV y el matched documents son los clean_Job_Description.
#Cuando hacemos un transform estamos pasandole la query para que se fije los TF-IDF que tiene ya entrenados
#del modelo (cuando hicimos fit ya lo entrenamos... y lo hicimos con los cvs y los job descriptions). 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances as pcd

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_vectorizer.fit(df_Jobs_and_Candidates['clean_Content_CV'] + " " + df_Jobs_and_Candidates['clean_Job_Description'])

A = tfidf_vectorizer.transform(df_Jobs_and_Candidates['clean_Content_CV'])
B = tfidf_vectorizer.transform(df_Jobs_and_Candidates['clean_Job_Description'])

# compute and the cosine similarity:
cosine = 1 - pcd(A, B)

df_Jobs_and_Candidates['tfidf_cosine'] = cosine

df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,tfidf_cosine,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",0.058337,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
1,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",0.099616,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
2,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",0.028673,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",0.020506,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp..."
4,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",0.051409,Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ..."
...,...,...,...,...,...,...,...,...,...
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",0.074330,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",0.081982,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",0.051793,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",0.100100,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."


In [84]:
#SI se agrega un Candidato nuevo, se agrega al DF y hago que me devuelva la columna en la que se insertó. 
#Y con esa columna hago el cálculo que hice arriba.. por ejemplo ahora para la de indice 96:
A = clf.transform([df_Jobs_and_Candidates['clean_Content_CV'][96]])
B = clf.transform([df_Jobs_and_Candidates['clean_Job_Description'][96]])

cosine = 1 - pcd(A, B)
cosine
#df_Jobs_and_Candidates['tfidf_cosine'] = cosine

array([0.08198229])

## 1.4- Realizando Comparaciones.

### 1.4.1- Usamos el Word2vec descargado.

In [24]:
#Link del cual descargamos el archivo: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
EMBEDDING_FILE = '/home/fedricio/Desktop/Embeddings_Utilizados/Word2vec/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

###  1.4.2- Funciones necesarias para Cosine Sim y WDM

In [25]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

def get_cosine(doc1, doc2):
    #Get the cosine similarity between two documents.
    #Depends on the angle between two non zero vectors which are constructed by each word frequency in the two documents.
        #input  --> doc1: the first document as STRING.
                    #doc2: the second document as STRING.
        #output --> cosine similarity score

    vec1 = text_to_vector([doc1])
    vec2 = text_to_vector([doc2])
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def WMD(doc1,doc2):
      #Preprocess the document first and remove english stopwords then call the function that calculates the word mover distance
      #input --> doc1: the first document as STRING.
                 #doc2: the second document as STRING.
      #output --> Word Mover's Distance score
    #print(doc1)
    first_doc = tokenize([doc1])[0]     #[0] porque es una lista dentro de otra lista.
    #print(first_doc)
    second_doc = tokenize([doc2])[0]    #[0] porque es una lista dentro de otra lista.
    return (word2vec.wmdistance(first_doc, second_doc))  #'wmdistance' return the word mover distance between two documents. 

###  1.4.3- Comparaciones.

Las comparaciones a realizar son:

    1-Entre el contenido de los CVs y las descripciones de los puestos.
    2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.
    3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" /personalizadas.

#### 1.4.3.1-Entre el contenido de los CVs y las descripciones de los puestos.

In [27]:
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords
2,Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co..."
7,HCM Consultant 2,4+ years of experience in peoplesoft or oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp...","*, project, management, &, experience, stakeho..."
0,Machine Learning Engineer,leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ...","experience, projects, leveraging, latest, tech..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora..."
8,Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo..."
4,Web Developer Full Stack,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,..."
5,Web Developer Full Stack 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ...","5, (required), enter, existing, website, codeb..."


In [28]:
df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
6,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
2,Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
4,HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co..."
7,HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
0,ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
8,Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
3,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
5,Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."
9,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."


###### Creamos un nuevo DF concatenando los 2 dataframe de manera que se comparen todos los CVs con todos los Jobs:

In [ ]:
#Nos quedarán 10 x 10 = 100 filas.
df_Jobs_and_Candidates = pd.merge(df_Candidates.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

In [63]:
#Como prueba obtenemos el WMD y Cosine Sim comparando para la 1ra fila (entre las columnas Content_CV y Job_Description)
first_job_descr = df_Jobs_and_Candidates.loc[0,'clean_Job_Description']
first_content_cv = df_Jobs_and_Candidates.loc[0,'clean_Content_CV']

cosine_result = round(get_cosine(first_job_descr,first_content_cv),3)
wmd_result = round(WMD(first_job_descr,first_content_cv),3)
print(cosine_result)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  #similarity = 1 / (1 + distance) https://groups.google.com/g/gensim/c/-pRZnsOEaPQ 
print(similarity_wdm)

NameError: name 'df_Jobs_and_Candidates' is not defined

In [32]:
#Aplicamos las funciones get_cosine y WMD para TODO el DF... entre el contenido del CV y la descripcion del 
#puesto; y el resultado lo guardamos en  las columnas 'Cosine_Job_Desc' y 'WMD_Job_Desc'.

import time

inicio_cosine = time.time()
df_Jobs_and_Candidates['Cosine_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['clean_Content_CV'],row['clean_Job_Description']),3), axis=1)
fin_cosine = time.time()
print(fin_cosine-inicio_cosine)

inicio_WMD = time.time()
#df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Job_Description']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['clean_Content_CV'],row['clean_Job_Description'])))),3), axis=1)
fin_WMD = time.time()
print(fin_WMD-inicio_WMD)

df_Jobs_and_Candidates

0.46358704566955566
116.4919548034668


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords,Cosine_Job_Desc,WMD_Job_Desc
0,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel...",0.139,0.489
1,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ...",0.210,0.494
2,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co...",0.085,0.480
3,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant 2,4+ years of experience in peoplesoft or oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp...","*, project, management, &, experience, stakeho...",0.058,0.475
4,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Machine Learning Engineer,leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ...","experience, projects, leveraging, latest, tech...",0.118,0.485
...,...,...,...,...,...,...,...,...,...,...,...
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora...",0.158,0.496
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o...",0.154,0.501
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo...",0.115,0.486
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Web Developer Full Stack,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,...",0.188,0.490


In [34]:
is_Bradly_Johnston =  df_Jobs_and_Candidates['Candidate_Name']=='Security_Specialist_Denis Banik'
new_DF = df_Jobs_and_Candidates[is_Bradly_Johnston]
print(new_DF.shape)
new_DF

(10, 11)


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords,Cosine_Job_Desc,WMD_Job_Desc
70,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Data Scientist,"master’s degree or above in a stem field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...","master’s, degree, or, above, in, a, stem, fiel...",0.088,0.491
71,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Data Scientist 2,"\nreporting to the director, data & analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...","projects, techniques, skills, organization’s, ...",0.103,0.490
72,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",HCM Consultant,\nthe oracle cloud hcm absence consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...","requirements, experience, work, consulting, co...",0.108,0.490
73,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",HCM Consultant 2,4+ years of experience in peoplesoft or oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp...","*, project, management, &, experience, stakeho...",0.127,0.493
74,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Machine Learning Engineer,leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ...","experience, projects, leveraging, latest, tech...",0.169,0.500
75,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...","2+, knowledge, of, with, experience, collabora...",0.131,0.494
76,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...","experience, with, work, fast-paced, combine, o...",0.345,0.507
77,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...","security, of, handling, incoming, requests, fo...",0.349,0.504
78,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu...",Web Developer Full Stack,\n\n graduate degree in information technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...","experience, written, typescript), skills, and,...",0.128,0.486
79,Security_Special

In [40]:
#Exportamos el csv:
df_Jobs_and_Candidates.to_csv('DF_Exportado_Jobs_And_Candidates.csv',index = False)
#Si lo queremos importar:
DF_J_and_C = pd.read_csv('DF_Exportado_Jobs_And_Candidates.csv')
DF_J_and_C.tail()

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,Keywords,Cosine_Job_Desc,WMD_Job_Desc
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"['karen', 'higgins', 'b', 'developer', 'area',...",Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...","2+, knowledge, of, with, experience, collabora...",0.158,0.496
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"['karen', 'higgins', 'b', 'developer', 'area',...",Security Specialist,work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"['work', 'fast-paced', 'environment', 'combine...","experience, with, work, fast-paced, combine, o...",0.154,0.501
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"['karen', 'higgins', 'b', 'developer', 'area',...",Security Specialist 2,\n handling incoming requests for assistanc...,handling incoming requests assistance business...,"['handling', 'incoming', 'request', 'assistanc...","security, of, handling, incoming, requests, fo...",0.115,0.486
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"['karen', 'higgins', 'b', 'developer', 'area',...",Web Developer Full Stack,\n\n graduate degree in information technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...","experience, written, typescript), skills, and,...",0.188,0.490
99,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"['karen', 'higgins', 'b', 'developer', 'area',...",Web Developer Full Stack 2,\n· enter existing website codebases and exten...,enter existing website codebases extend functi...,"['enter', 'existing', 'website', 'codebases', ...","5, (required), enter, existing, website, codeb...",0.204,0.507


#### 1.4.3.2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.

In [ ]:
#APlicamos las funciones get_cosine y WMD entre el contenido del CV y las keywords que obtuvimos previamente 
#del puesto y el resultado lo guardamos en  las columnas 'Cosine_Job_Keywords' y 'WMD_Job_Keywords'.
df_Jobs_and_Candidates['Cosine_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['clean_Content_CV'],row['Keywords']),3), axis=1)
#df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Keywords']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['clean_Content_CV'],row['Keywords'])))),3), axis=1)

df_Jobs_and_Candidates

#### 1.4.3.3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" / personalizadas.

In [89]:
Content_CV_Phoebe_Buffay = df_Jobs_and_Candidates.loc[17,'clean_Content_CV']
print(Content_CV_Phoebe_Buffay)

karen higgins b developer areas expertise personal summary ambitious problem solver passion online businesses search engine optimisation would like join team like-minded developers karen much experience creating logical innovative solutions complex problems identifying technical thorough precise everything keen interest solutions technology mobile applications user experience someone takes responsibility personal development continually evaluating client facing skills upgrading skills stays cutting edge web development natural problem solver proven successfully completing projects consultancies software houses web troubleshooting design agencies departments javascript right karen looking suitable position company outstanding performance recognised work variety web graphi cs high profile projects bug fixing career history web design company - birmingham building tem plates web developer jul - present image cropping responsible working range projects designing appealing websites interact

In [32]:
#Agregamos 2 filas más al DF con las keywords personalizadas (las segundas keywords no tendrán nada que ver
#al contenido del CV). Y luego aplicamos sim cosine y WDM para estos casos y vemos los resultados.

df_Jobs_and_Candidates.loc[18] = ['Test',Content_CV_Phoebe_Buffay,'','','data science, machine learning, pandas, python, sql','','','',''] 
#Es lo mismo poner las Keywords con o sin comas (da igual el resultado, ya que el procesamiento se encarga de esto). 
df_Jobs_and_Candidates.loc[19] = ['Test',Content_CV_Phoebe_Buffay,'','','football, tennis, spanish','','','',''] 

df_Jobs_and_Candidates.tail()

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
#TEST: Usamos la funcion text_to_vector() para las keywords:
vector_test_description = text_to_vector(df_Jobs_and_Candidates.iloc[18]['Keywords'])
vector_test_description

#Ahora si aplicamos sim cosine y WDM para estos casos y vemos los resultados. 
df_Jobs_and_Candidates.loc[18,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)

#df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords'])))),3)

df_Jobs_and_Candidates.loc[19,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)

#df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords'])))),3)

df_Jobs_and_Candidates.tail()

## 1.5- Utilizando glove en lugar de word2vec para aplicar WDM.

In [33]:
# Load Word Embedding Model
import gensim
from gensim.models.keyedvectors import KeyedVectors

print('gensim version: %s' % gensim.__version__)
#glove_model = gensim.models.KeyedVectors.load_word2vec_format('../model/text/stanford/glove/glove.6B.50d.vec')
glove_model = KeyedVectors.load_word2vec_format('/home/fedricio/Desktop/Embeddings_Utilizados/Glove/glove.6B.50d.txt', binary=False, no_header=True)

gensim version: 4.0.0


In [36]:
CV_text = df_Jobs_and_Candidates.loc[32,'clean_Content_CV']
CV_text

'calonge federico matias oracle consultant page hcm technical consultant federico ha working oracle tools since october march participated erp cloud projects performing reporting tasks working modules ap ar gl since march participating hcm cloud projects performing reporting extractions integrations tasks working modules core nd recruitment ince october h e member artificial intelligence ai committee oracle strong interest data science machine learning last year computer engineering degree studying th year english personal ac ademic programming projects https github com federicocalonge skills summary oracle hcm cloud core recruitment oracle erp cloud accounts receivable ar accounts payable ap general ledger gl main experience points reporting sql bi publisher intermediate - advanced technical documentation extractions api rest hcm extract web services soap basic - intermediate integrations hdl basic - intermediate extra data analysis implementation machine learning algorithms python in

In [65]:
CV_text = df_Jobs_and_Candidates.loc[0,'clean_Content_CV']
CV_text_2 = df_Jobs_and_Candidates.loc[11,'clean_Content_CV']
CV_text_3 = df_Jobs_and_Candidates.loc[32,'clean_Content_CV']
Job_Keyw_text = "bsc msc phd physics mathematics biomedical software engineering computer science years experience developing image processing machine learning products pipelines advantage familiarity web development e g flask serverless deployment e g docker cloud services e g aws entrepreneurial experience passion working small teams peer-reviewed academic publications"
Job_Keyw_text_2 = "football tennis spanish"
texts_to_compare = [Job_Keyw_text,CV_text,CV_text_2,CV_text_3,Job_Keyw_text_2]
texts_tokens = tokenize(texts_to_compare)   #Función definida en las secciones anteriores. 
print(texts_tokens)

[['bsc', 'msc', 'phd', 'physics', 'mathematics', 'biomedical', 'software', 'engineering', 'computer', 'science', 'years', 'experience', 'developing', 'image', 'processing', 'machine', 'learning', 'products', 'pipelines', 'advantage', 'familiarity', 'web', 'development', 'e', 'g', 'flask', 'serverless', 'deployment', 'e', 'g', 'docker', 'cloud', 'services', 'e', 'g', 'aws', 'entrepreneurial', 'experience', 'passion', 'working', 'small', 'teams', 'peerreviewed', 'academic', 'publications'], ['chandler', 'bing', 'chandler', 'bing', 'vit', 'ac', 'education', 'l', 'date', 'birth', 'may', 'languages', 'known', 'hindi', 'english', 'spanish', 'coursework', 'machine', 'learning', 'data', 'visualization', 'perating', 'systems', 'database', 'management', 'system', 'discrete', 'maths', 'skills', 'c', 'java', 'python', 'arduino', 'android', 'html', 'php', 'javascript', 'mysql', 'r', 'studio', 'keras', 'nltk', 'parser', 'sql', 'experience', 'android', 'club', 'coordinator', 'front', 'end', 'develope

In [63]:
texts_tokens[1]   #CV_text

['chandler',
 'bing',
 'chandler',
 'bing',
 'vit',
 'ac',
 'education',
 'l',
 'date',
 'birth',
 'may',
 'languages',
 'known',
 'hindi',
 'english',
 'spanish',
 'coursework',
 'machine',
 'learning',
 'data',
 'visualization',
 'perating',
 'systems',
 'database',
 'management',
 'system',
 'discrete',
 'maths',
 'skills',
 'c',
 'java',
 'python',
 'arduino',
 'android',
 'html',
 'php',
 'javascript',
 'mysql',
 'r',
 'studio',
 'keras',
 'nltk',
 'parser',
 'sql',
 'experience',
 'android',
 'club',
 'coordinator',
 'front',
 'end',
 'developer',
 'technovit',
 'website',
 'team',
 'key',
 'entertainer',
 'friends',
 'projects',
 'good',
 'females',
 'android',
 'application',
 'w',
 'omen',
 'health',
 'safety',
 'emergency',
 'mode',
 'using',
 'android',
 'modules',
 'safest',
 'route',
 'using',
 'image',
 'processing',
 'disease',
 'prediction',
 'using',
 'svm',
 'linear',
 'regression',
 'face',
 'detection',
 'recognition',
 'using',
 'openmp',
 'opencv',
 'openmp',
 'cu

In [66]:
#Imprimimos las distancias entre Job_Keyw_text y los 3 CVs:

tokens_job_keyw = texts_tokens[0]
tokens_CV_1 = texts_tokens[1]
tokens_CV_2 = texts_tokens[2]
tokens_CV_3 = texts_tokens[3]
tokens_job_keyw_2 = texts_tokens[4]

distance_1 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_1)
print('Distance with CV_1= %.4f' % distance_1)

distance_2 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_2)
print('Distance with CV_2= %.4f' % distance_2)

distance_3 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_3)
print('Distance with CV_3= %.4f' % distance_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
distance_4 = glove_model.wmdistance(tokens_job_keyw_2, tokens_CV_1)
print('Distance keywords_2 with CV_1= %.4f' % distance_4)

print('#'*40)
############################################################
#Ahora aplicamos lo de --> similarity = 1 / (1 + distance):

sim_1 = round((1 / (1 + distance_1)),3)
print('Similarity with CV_1= %.4f' % sim_1)

sim_2 = round((1 / (1 + distance_2)),3)
print('Similarity with CV_2= %.4f' % sim_2)

sim_3 = round((1 / (1 + distance_3)),3)
print('Similarity with CV_3= %.4f' % sim_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
sim_4 = round((1 / (1 + distance_4)),3)
print('Similarity keywords_2 with CV_1= %.4f' % sim_4)

Distance with CV_1= 0.8981
Distance with CV_2= 0.8440
Distance with CV_3= 0.8419
Distance keywords_2 with CV_1= 1.2049
########################################
Similarity with CV_1= 0.5270
Similarity with CV_2= 0.5420
Similarity with CV_3= 0.5430
Similarity keywords_2 with CV_1= 0.4540


## 1.6- Utilizando spacy en lugar de word2vec o glove para aplicar WDM.

In [40]:
#Ejemplo de la doc oficial modificado: doc oficial: https://spacy.io/universe/project/wmd-relax
#Para esto descargue spacy: >conda install -c conda-forge spacy
#Y el modelo: python -m spacy download en_core_web_lg
import spacy
import wmd

nlp = spacy.load('en_core_web_lg', create_pipeline=wmd.WMD.create_spacy_pipeline)

In [41]:
#Ejemplo:
doc1 = nlp("Politician speaks to the media in Illinois.")
doc2 = nlp("The president greets the press in Chicago.")
doc3 = nlp("The president greets the press in Chicago.")
print(doc1.similarity(doc2))
print(doc2.similarity(doc3))

0.8705018149808114
1.0


In [44]:
#Nuestro caso:

#Preprocesamos el doc_1 a comparar (el doc_2 no hace falta porque son keywords):
doc_1 = df_Jobs_and_Candidates.loc[19,'clean_Content_CV']
doc_1_tokenize = tokenize([doc_1])              #Usando la funcion definida en las secciones previas.
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    #Pasamos la lista a string.
new_text_doc_1                                  #Nuevo texto sin espacios, stop words y demas (ver todo lo que hace 'tokenize')

doc1 = nlp(new_text_doc_1)
doc2 = nlp('data science, machine learning, pandas, python, sql')
print(doc1.similarity(doc2))

0.7440745766755973


In [45]:
#Hacemos lo mismo que arriba solo que ahora nuestras keywords serán otras que no tendrán que ver con el CV:

doc_1 = df_Jobs_and_Candidates.loc[19,'clean_Content_CV']
doc_1_tokenize = tokenize([doc_1])              
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    
new_text_doc_1                                 

doc1 = nlp(new_text_doc_1)
doc2 = nlp('football tennis spanish')  #CAMBIAMOS ESTO.
print(doc1.similarity(doc2))

0.45084626580079523
